# Thesis Outcomes

In this notebook I will compare the result I got from all the algorithms I implemented. 

I will start by comparing both packing methods with each other. Then i will be comparing the Mean, followed by the Inner Product, and finish with the Variance.

For all the algorithms I am working with vector of size n = 8192 which are always filled with exactly 8192 values which can be 1, 2 or 3.

## Setup

In [9]:
# Imports
import os
import glob
import pandas as pd

In [12]:
# Function to extract files from folder
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
            files = glob.glob(os.path.join(root, '*.csv'))
            for f in files:
                    all_files.append(os.path.abspath(f))
    return all_files

In [33]:
# Get file names
outcomes_CSV = get_files("./timeCSVs")

# Import csv as a Dataframe
print(outcomes_CSV[0])
inner_Product_DF = pd.read_csv(outcomes_CSV[0])
print(outcomes_CSV[1])
mean_DF = pd.read_csv(outcomes_CSV[1])
print(outcomes_CSV[2])
variance_DF = pd.read_csv(outcomes_CSV[2])

c:\Users\berna\Desktop\Personal\FEUP\Thesis\Project\HE_Optimization\Outcomes\timeCSVs\innerProduct.csv
c:\Users\berna\Desktop\Personal\FEUP\Thesis\Project\HE_Optimization\Outcomes\timeCSVs\mean.csv
c:\Users\berna\Desktop\Personal\FEUP\Thesis\Project\HE_Optimization\Outcomes\timeCSVs\variance.csv


## Coefficient packing vs Slot packing

In [35]:
# Join all the times from each packing into a DataFrame

### Get all the coef times
coef_DF = pd.concat([inner_Product_DF[inner_Product_DF['type'].str.contains('coef')], 
                        mean_DF[mean_DF['type'].str.contains('coef')], 
                        variance_DF[variance_DF['type'].str.contains('coef')]]
                        , ignore_index=True)

### Get all the slot times
slot_DF = pd.concat([inner_Product_DF[inner_Product_DF['type'].str.contains('slot')], 
                        mean_DF[mean_DF['type'].str.contains('slot')], 
                        variance_DF[variance_DF['type'].str.contains('slot')]]
                        , ignore_index=True)

,type,setup,encryption,homomorphic,decryption,plaintext,total,value
count,2015,2015,2015,2015,2015,2015,2015,1015.0
unique,4,122,55,93,11,50,104,2.0
top,optimized-slot,224,-19,72,0,0,13651,2.0
freq,1000,146,378,235,1966,1015,1000,1005.0


## Mean

For the mean I have 4 types of implementation.

The first, and the most naive, is the one where each value is encrypted into a single ciphertext. 

The second method is where we encrypt only one ciphertext and calculate the total sum by doing n rotations.

The third method is where we encrypt only one ciphertext and calculate the total sum by doing log2(n) rotations.

The forth method is only applicable to Coefficient Packing because it uses a pre-processing technique that only works with these type of packing.

In [47]:
mean_DF['type'].unique()

array(['simple-slot', 'simple-coef', 'rot-slot', 'opt-rot-slot',
       'opt-coef-rot', 'pre-proc-coef-rot'], dtype=object)

In [48]:
mean_DF[mean_DF['type'] == "opt-rot-slot"]

,type,setup,encryption,homomorphic,decryption,plaintext,total,value
20,opt-rot-slot,280,12,72,1,0,365,2.0
21,opt-rot-slot,283,12,70,1,0,366,2.0
22,opt-rot-slot,284,11,74,1,0,370,2.0
23,opt-rot-slot,287,12,84,1,0,384,2.0
24,opt-rot-slot,281,12,73,1,0,367,2.0
...,...,...,...,...,...,...,...,...
1015,opt-rot-slot,279,11,71,1,0,362,2.0
1016,opt-rot-slot,289,11,74,1,0,375,2.0
1017,opt-rot-slot,281,12,75,1,0,369,2.0
1018,opt-rot-slot,297,11,71,1,0,380,2.0


In [53]:
### Create DF for each type of implementation
simple_DF = mean_DF[mean_DF['type'].str.contains('simple')]
opt_rot_DF = pd.concat([
    mean_DF[mean_DF['type'] == "opt-rot-slot"], 
    mean_DF[mean_DF['type'] == "opt-coef-rot"]
    ])
rot_DF = pd.concat([
    mean_DF[mean_DF['type'] == "rot-slot"], 
    mean_DF[mean_DF['type'] == "coef-rot"]
    ], ignore_index=True)
pre_proc_DF = mean_DF[mean_DF['type'].str =='pre-proc-coef']

implementation_dfs = {'simple': simple_DF, 'optimized': opt_rot_DF, 'rotation': rot_DF, 'pre_processed': pre_proc_DF}

KeyError: False

### Total Time Comparison

In [ ]:
implementation_averages = {}

for name, df in implementation_dfs.items():
    implementation_averages[name] = df[' total'].mean()

implementation_averages

### Setup

### Encryption

### Homomorphic Operations

### Conclusion

The first is clearly the slowest and it makes sense. We end up having to encrypt n ciphertexts which makes this method incredibly inefficient.